In [ ]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt 
import cv2
import random
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc
import tensorflow as tf
from tensorflow import keras
import os

In [ ]:
path_df = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if('images' in dirname):
            path_df.append(os.path.join(dirname, filename))
            
random.shuffle(path_df)

In [ ]:
def remove_zero_pad(image):
    dummy = np.argwhere(image != 0) 
    max_y = dummy[:, 0].max()
    min_y = dummy[:, 0].min()
    min_x = dummy[:, 1].min()
    max_x = dummy[:, 1].max()
    crop_image = image[min_y:max_y, min_x:max_x]

    return crop_image

def ROC_Curve(y_test, prob_pred):
    binarizer = LabelBinarizer()
    y_test_bin = binarizer.fit_transform(y=y_test)

    y_score = prob_pred

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(4):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(4)]))

    mean_tpr = np.zeros_like(all_fpr)
    for i in range(4):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    mean_tpr /= 4  

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    plt.figure(figsize=(14,12))

    for i in range(4):
        plt.plot(fpr[i],tpr[i],label="ROC curve of class {0} (area = {1:0.2f})".format(encoder.inverse_transform([i])[0], roc_auc[i]))

    plt.plot(fpr["micro"],tpr["micro"],label="micro-average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),linestyle="--",)

    plt.plot(fpr["macro"],tpr["macro"],label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),linestyle="--",)

    plt.plot([0, 1], [0, 1], "k--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

In [ ]:
labels = ['COVID','Lung_Opacity','Normal','Viral Pneumonia']
for i in range(4):
    fig, axs = plt.subplots(1,5, figsize = (50,10))
    print('X-Ray Images of Class',labels[i],':-')
    paths = [k for k in path_df if(labels[i] in k)]
    for j in range(5):
        img = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(paths[j], color_mode = 'grayscale'))
        axs[j].imshow(img, cmap='gray')
    plt.show()
    
X_train = []
X_val = []
X_test = []
y_train = []
y_val = []
y_test = []
for i in path_df[0:int(len(path_df)*0.7)]:
    img = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(i, color_mode = 'grayscale'))
    crop_img = remove_zero_pad(img)
    img = tf.image.resize(crop_img, (175, 175))
    X_train.append(np.array(img/255.0, dtype = np.float16))
    y_train.append(i.split('/')[-3])    

In [ ]:
for i in path_df[int(len(path_df)*0.7):int(len(path_df)*0.8)]:
    img = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(i, color_mode = 'grayscale'))
    crop_img = remove_zero_pad(img)
    img = tf.image.resize(crop_img, (175, 175))    
    X_val.append(np.array(img/255.0, dtype = np.float16))
    y_val.append(i.split('/')[-3])
    
for i in path_df[int(len(path_df)*0.8):int(len(path_df))+1]:
    img = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(i, color_mode = 'grayscale'))
    crop_img = remove_zero_pad(img)
    img = tf.image.resize(crop_img, (175, 175))
    X_test.append(np.array(img/255.0, dtype = np.float16))
    y_test.append(i.split('/')[-3])

In [ ]:
print('Training images count:', len(y_train))
print('Validation images count:', len(y_val))
print('Test images count:', len(y_test))

value_count_df = pd.DataFrame(pd.Series(np.concatenate([y_train, y_val, y_test])).value_counts()).rename_axis('unique_values').reset_index()
value_count_df.columns = ['Class','Count']
plt.bar(value_count_df['Class'], value_count_df['Count'])
plt.show()

encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_val = encoder.fit_transform(y_val)
y_test = encoder.fit_transform(y_test)

X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

In [ ]:
def ResNet_CNNModel():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(512, 7, strides=2, input_shape=np.shape(X_train[0]), padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("selu"))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Conv2D(128, 3, strides=1, padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("selu"))
    model.add(keras.layers.MaxPool2D(pool_size=3, strides=1, padding="same"))

    prev_filters = 128 
    for filters in [128]*2 + [64]*2:                    
        strides = 1 if filters == prev_filters else 2
        model.add(ResidualUnit(filters, strides=2))
        prev_filters = filters

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128,activation = 'selu'))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(64,activation = 'selu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(4, activation="softmax"))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer = keras.optimizers.Adam(), metrics=['accuracy'])         #"Adam" optimizer is used
    
    return model

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="selu", **kwargs):   
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)                     
        self.main_layers = [                                                    
            keras.layers.Conv2D(filters, 3, strides=strides,padding="same"),    
            keras.layers.BatchNormalization(),                                  
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1,padding="same"),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1: 
            self.skip_layers = [                                                
                keras.layers.Conv2D(filters, 1, strides=strides,padding="same"),
                keras.layers.BatchNormalization()]
        
    def call(self, inputs):                                                     
        Z = inputs
        for layer in self.main_layers:    
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)                                      
    
    
ResNetmodel = ResNet_CNNModel()
print('Summary of ResNet CNN Model:-')
ResNetmodel.summary()

checkpoint_path_resnet = 'training_1/resnet.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path_resnet)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path_resnet,                          
                                                save_best_only=True,
                                                save_weights_only=True,
                                                mode = 'max',
                                                monitor = 'val_accuracy',
                                                verbose=1)

ResNetmodel_hist = ResNetmodel.fit(X_train, y_train, epochs = 30, batch_size = 25, validation_data = (X_val, y_val), callbacks = [cp_callback])

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (20,7))
plt.title("ResNet Model")
axs[0].plot(np.arange(1,31,1), ResNetmodel_hist.history['accuracy'], label = 'Accuracy')
axs[0].plot(np.arange(1,31,1), ResNetmodel_hist.history['val_accuracy'], label = 'Val Accuracy')
axs[0].legend()
axs[0].grid()

axs[1].plot(np.arange(1,31,1), ResNetmodel_hist.history['loss'], label = 'Loss')
axs[1].plot(np.arange(1,31,1), ResNetmodel_hist.history['val_loss'], label = 'Val Loss')
axs[1].legend()
axs[1].grid()

plt.show()

In [ ]:
prob_pred_ResNet = ResNetmodel.predict(X_test)
y_pred_ResNet = np.argmax(prob_pred_ResNet, axis=1)

print('Accuracy on Test Set:',accuracy_score(y_test, y_pred_ResNet))
print('Classification Report:-')
print(classification_report(encoder.inverse_transform(y_test), encoder.inverse_transform(y_pred_ResNet)))
print('Classwise ROC Curve for ResNet Model :-')
ROC_Curve(y_test, prob_pred_ResNet)

In [ ]:
ResNetModel_New = ResNet_CNNModel()
ResNetModel_New.load_weights(checkpoint_path_resnet)

new_prob_pred_resnet = ResNetModel_New.predict(X_test)
y_pred_new = np.argmax(new_prob_pred_resnet, axis=1)
print('Accuracy on Test Set:',accuracy_score(y_test, y_pred_new))
print('Classification Report:-')
print(classification_report(encoder.inverse_transform(y_test), encoder.inverse_transform(y_pred_new)))
print('Classwise ROC Curve for ResNet Model :-')
ROC_Curve(y_test, new_prob_pred_resnet)